<a href="https://colab.research.google.com/github/ParkSoeon/ParkSoeon/blob/main/%5BML_Assignment2%5DLogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Task**
1. Logistic Regression Implementation
  - Sigmoid(Logistic Output activate)
  - Write Cost Function
  - Function that calculate graident of parameter for cost(Used for Gradient Descent)
2. Dataset(Given)
3. Train and Evaluation
  - Train Logistic Regression Model with train data
  - Evaluate model performance with test data
    <br> (with accuracy and confusion matrix)
4. Dicision Boundary Analysis
  - Plot the decision boundary on the training set to visulize the model's performance

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# convert txt data to csv format for data1
import csv
import itertools

with open('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data1.txt', 'r') as in_file:
  stripped = (line.strip() for line in in_file)
  lines = (line.split(',') for line in stripped if line)
  with open('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data1.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('column1', 'column2', 'column3'))
    writer.writerows(lines)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data1.csv')
df.head()

,column1,column2,column3
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   column1  100 non-null    float64
 1   column2  100 non-null    float64
 2   column3  100 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.5 KB


In [ ]:
df.column3.value_counts()

,count
column3,
1,60
0,40


In [ ]:
# convert txt data to csv format for data2
import csv
import itertools

with open('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data2.txt', 'r') as in_file:
  stripped = (line.strip() for line in in_file)
  lines = (line.split(',') for line in stripped if line)
  with open('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data2.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('column1', 'column2', 'column3'))
    writer.writerows(lines)

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/24-2/Major/MachineLearning_LiliaTightiz/Assignments/Assignment2/data/ex2data2.csv')
df2.head()

,column1,column2,column3
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   column1  118 non-null    float64
 1   column2  118 non-null    float64
 2   column3  118 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.9 KB


In [ ]:
df2.column3.value_counts()

,count
column3,
0,60
1,58


---

In [ ]:
# Split data as train/test dataset

# Train
x = df.drop('column3', axis = 1)

# Target
y = df['column3']

# Split [Train:Test] = [70:30]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((70, 2), (30, 2), (70,), (30,))

'x_train' and 'x_test': 2-Dimensional<br>
while, 'y_train' and 'y_test' are 1D<br>
### **→ Need to Match Tensors**

In [ ]:
# Convert y_train and y_test to 2D
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [ ]:
# Task 1: Implement Sigmoid
def sigmoid(z):
  # there is also a given library from *torch.nn.functional*
  ### START CODE HERE ###
  s = 1/(1+np.exp(-z))
  ### END CODE HERE ###
  return s

# Task 2: Write Cost Function
def cost(X, y, w, b):
  m = X.shape[0]

  z = np.dot(X, w) + b
  h = sigmoid(z)

  ### START CODE HERE ###
  epsilon = 1e-8
  cost = (-1/m) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1-h + epsilon))
  ### END CODE HERE ###

  return cost

# Task 3: Gradient Descent
def gradient(X, y, w, b):
  m = X.shape[0]

  z = np.dot(X, w) + b
  h = sigmoid(z)
  # Slope
  dw = (1/m) * np.dot(X.T, (h - y))
  db = (1/m) * np.sum(h - y)

  return dw, db

In [ ]:
# Train Model Function
def train_model(X, y, w, b, alpha, epochs):
  costs = []

  for i in range(epochs):
    dw, db = gradient(X, y, w, b)

    # Parameters Update(GD)
    w -= alpha * dw
    b -= alpha * db

    cost_val = cost(X, y, w, b)
    costs.append(cost_val)

    if i % 100 == 0:
      print(f"Epoch: {i}, Cost: {cost_val}")

  return w, b, costs

In [ ]:
w = np.zeros((x_train.shape[1], 1)) # Initialize Weights
b = 0 # Initialize Interception
alpha = 0.00001
epochs = 1000

# Train Model
w, b, costs = train_model(x_train, y_train, w, b, alpha, epochs)

Epoch: 0, Cost: 0.6901308701671358
Epoch: 100, Cost: 0.6254881733034123
Epoch: 200, Cost: 0.6245246495237897
Epoch: 300, Cost: 0.6245004003785833
Epoch: 400, Cost: 0.624494210014872
Epoch: 500, Cost: 0.6244885965289688
Epoch: 600, Cost: 0.6244831820355988
Epoch: 700, Cost: 0.624477930593294
Epoch: 800, Cost: 0.6244728173503984
Epoch: 900, Cost: 0.6244678213385131


**Trial 1.**
```
alpha = 0.01
epochs = 1000
```

Epoch: 0, Cost: 5.1406520183899085<br>
Epoch: 100, Cost: 6.9496118485317595<br>
Epoch: 200, Cost: 0.8347027262818276<br>
Epoch: 300, Cost: 7.228761823668115<br>
Epoch: 400, Cost: 7.10511959360144<br>
Epoch: 500, Cost: 3.726054625324725<br>
Epoch: 600, Cost: 7.105119631031701<br>
Epoch: 700, Cost: 6.773915636246067<br>
Epoch: 800, Cost: 7.105054824123828<br>
Epoch: 900, Cost: 5.65224357348397<br>
<br>
→ Accuracy on Train Data: 0.38571428571428573
<br><br>
**Trial 2.**
```
alpha = 0.001
epochs = 1000
```
Epoch: 0, Cost: 0.6988217178913696<br>
Epoch: 100, Cost: 0.6240647549385432<br>
Epoch: 200, Cost: 0.6236314829767015<br>
Epoch: 300, Cost: 0.6231989441273678<br>
Epoch: 400, Cost: 0.6227670588928278<br>
Epoch: 500, Cost: 0.6223358263646196<br>
Epoch: 600, Cost: 0.6219052456345826<br>
Epoch: 700, Cost: 0.6214753157944738<br>
Epoch: 800, Cost: 0.6210460359359741<br>
Epoch: 900, Cost: 0.6206174051506979<br>
<br>
→ Accuracy on Train Data: 0.6142857142857143
<br><br>
**Trial 3.**
```
alpha = 0.0001
epochs = 1000
```
Epoch: 0, Cost: 0.6661130584577182<br>
Epoch: 100, Cost: 0.6244624683802956<br>
Epoch: 200, Cost: 0.6244166031276221<br>
Epoch: 300, Cost: 0.6243727306222963<br>
Epoch: 400, Cost: 0.6243292452156606<br>
Epoch: 500, Cost: 0.6242858392760928<br>
Epoch: 600, Cost: 0.6242424538636466<br>
Epoch: 700, Cost: 0.6241990776853368<br>
Epoch: 800, Cost: 0.6241557085768237<br>
Epoch: 900, Cost: 0.6241123461227097<br>
<br>
→ Accuracy on Train Data: 0.6142857142857143



In [ ]:
# Prediction Function
def predict(X, w, b):
  z = np.dot(X, w) + b
  h = sigmoid(z)

  y_pred = np.where(h >= 0.5, 1, 0)

  return y_pred

# Predict
y_pred_train = predict(x_train, w, b)
y_pred_test = predict(x_test, w, b)

In [ ]:
# Accuracy
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Accuracy on Train Data: {accuracy_train}")

accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on Test Data: {accuracy_test}")

Accuracy on Train Data: 0.6142857142857143
Accuracy on Test Data: 0.5666666666666667


In [ ]:
# Confusion Matrix(cm)
cm_train = confusion_matrix(y_train, y_pred_train)
print("Confusion Matrix on Train Data:")
print(cm_train)
print(" ")

cm_test = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix on Test Data:")
print(cm_test)

Confusion Matrix on Train Data:
[[ 0 27]
 [ 0 43]]
 
Confusion Matrix on Test Data:
[[ 0 13]
 [ 0 17]]
